# Cálculo de intervalo de confiança de acordo com o artigo "On the accuracy of simulated percentage points" (Juritz et al).

Nomenclatura usada no artigo:

- $f(x)$: pdf de uma variável aleatória $X$;
- $0 < p < 1$, ${\xi}_p$ denota o (100$p$) percentil de $X$;


## Estimativa do percentil e intervalo de confiança

Toda a discussão nessa seção é um resumo/tradução dos dados do artigo.

### The single-sample method

Nesse modelo o percentil é estimado a partir de uma única simulação de tamanho $n$.

Seja $X_{(i)}$ a $i$'ésima menor observação da simulação. De acordo com o artigo, David (1970) mostrou que para qualquer variável aleatória contínua o irtevalo $(X_{(r)}, X_{(s)})$, com $r < s$, abarca ${\xi}_p$ com probabilidade $ \pi(r, s, n, p) $. Essa probabilidade é dada por:


<center>$Prob(X_{(r)} \lt \xi_p \lt X_{(s)}) = \pi(r, s, n, p) = \sum_{i = 1}^{s - 1} \binom{n}{p} p^i (1 - p)^{(n-i)}$</center>


Note que o que está sendo feito aqui é:

1. fazemos uma simulação de Monte Carlo e o resultado são amostras da variável aleatória $X$; São realizadas $n$ simulações.

2. ordenamos o resultado do maior para o menor e chamamos de $X_{(i)}$ onde $ 1 <= i <= n $ é o índice dos resultados já ordenados;

3. estimamos o valor do (100$p$) percentil de $X$: ${\xi}_p \sim X_{(k)}$, onde $k = floor(np + 1)$.

4. calculamos dois índices, $r$ e $s$. Esses índices, quando aplicados em $X_{(i)}$, dão o intervalo de confiança para a nossa estimativa do percentil com probabilidade $\pi(r,s,n,p)$

$p$ é o percentil (ou quantil) que estamos interessados. Assim, $p = Prob(X \le \xi_p)$. $r$ e $s$ são **escolhidos** de tal forma que o intervalo tenha confiança de $ 1 - \alpha$, ou seja, $\pi(r,s,n,p) = 1 - \alpha$.

Considerando $n$ como um valor muito grande, $\pi(r,s,n,p)$ pode ser calculado aproximando a distribuação binomial por uma normal. Com isso, $r$ e $s$ pode ser encontrado como:

<center>
    $r = -w z_\gamma + np + \frac{1}{2}$ e $s = +w z_\gamma + np + \frac{1}{2}$
</center>

onde $w = \sqrt{np(1-p)}$ e $z_\gamma$ se refere ao (100$\gamma$) percentil da distribuição normal padrão. Por fim, $\gamma = 1 - \frac{\alpha}{2}$.

<span style="color:red">OBSERVAÇÃO: Essa aproximação é válida quando $np > 5$ e $nq > 5$ (onde $q = 1 - p$) [Referência](https://stats.libretexts.org/Courses/Las_Positas_College/Math_40%3A_Statistics_and_Probability/06%3A_Continuous_Random_Variables_and_the_Normal_Distribution/6.04%3A_Normal_Approximation_to_the_Binomial_Distribution). Como exemplo, se estamos trabalhando com o quantil 999, $ p = 0.999$ e $q = 0.001$, então $n$ deve ser no mínimo 5000.</span>


## Simulação com gausiana

Para testar, vamos considerar o cálculo do percentil 95 de uma distribuição normal. Ou seja, estamos interessados no cálculo de $\xi_{0.95}$. O valor real é de 1.6449. Vamos calcular o mesmo valor usando uma série de simulações diferentes valores de $n$ (número de amostras) e estimar o valor de $\xi_{0.95}$. Além disso, vamos fornecer um intervalo de confiança pra estimativa.

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

# Percentis desejados
p = np.array([0.95, 0.975])

# Quantidade de simulações
n = np.array([100, 400, 500, 1000, 5000, 10000, 100000])

# Índice na simulação de onde estarão os percentis.
# Note que estamos multiplicando as matrizes (qtd_percentis_simulados, 1) por (1, qtd_amostras_simuladas)
# O resultado (k) tem dimensão (qtd_percentis_simulados, qtd_amostras_simuladas).
# Ou seja, a dimensão 1 se refere aos percentis. Assim, k[i, :] vai nos fornecer todos os índices
# na amostra simulada para o percentil p[i]
# = np.floor(p.dot(n.T) + 1)

# Calcula os índices para r e s
# = 
#_gamma = p.dot(n.T)
#r = np.floor( -w * z_gamma + n * p + 0.5)
#s = np.floor( +w * z_gamma + n * p + 0.5)

# Seta a seed para 42 e gera uma amostra grande
#np.random.seed(42)
amostras = np.random.normal(size=np.max(n))

# Cálcula cada percentil para cada p e n

for p_i in p:
    print('----------------------------------------------------------')
    print(f'p = {p_i}. Valor real: {scipy.stats.norm.ppf(p_i)}')
    print('n\tk\tm_k\tr\tm_r\ts\tm_s\tCI Length')
    for n_i in n:
        # Extrai apenas os primeiros n_i dados da simulação maior e ordena do menor pro maior
        x = np.sort(amostras[0:n_i])
        # Índice da estimativa do percentil
        k_i = int(np.floor(p_i*n_i + 1))
        # Cálculo de w e z_gamma, necessários para o cálculo de r e s
        w = (n_i * p_i * (1 - p_i)) ** 0.5
        z_gamma = scipy.stats.norm.ppf(p_i)
        # Cálculo dos índices r e s (intervalos de confiança)
        r = int(np.ceil( -w * z_gamma + n_i * p_i + 0.5))
        s = int(np.ceil( +w * z_gamma + n_i * p_i + 0.5))
        # Imprime tabela
        m_r = f'{x[r-1]:.4f}' if r <= n_i else ' - '
        m_s = f'{x[s-1]:.4f}' if s <= n_i else ' - '
        ci_length = f'{(x[s-1] - x[r-1]):.3f}' if r <= n_i and s <= n_i else ' - '
        print(f'{n_i}\t{k_i}\t{x[k_i-1]:0.4f}\t{r}\t{m_r}\t{s}\t{m_s}\t{ci_length}')
        
        
#estimativa_percentil = amostras_ordenadas[k]

----------------------------------------------------------
p = 0.95. Valor real: 1.6448536269514722
n	k	m_k	r	m_r	s	m_s	CI Length
100	96	1.4646	92	0.9677	100	2.0992	1.131
400	381	1.5879	374	1.4646	388	1.8559	0.391
500	476	1.6381	468	1.5283	484	1.8669	0.339
1000	951	1.6103	940	1.5254	962	1.7068	0.181
5000	4751	1.6222	4726	1.5594	4776	1.6801	0.121
10000	9501	1.6519	9465	1.6191	9537	1.6908	0.072
100000	95001	1.6479	94888	1.6375	95114	1.6604	0.023
----------------------------------------------------------
p = 0.975. Valor real: 1.959963984540054
n	k	m_k	r	m_r	s	m_s	CI Length
100	98	1.5315	95	1.4180	102	 - 	 - 
400	391	1.9708	385	1.6381	397	2.3209	0.683
500	488	1.9893	482	1.8316	495	2.3209	0.489
1000	976	1.9141	966	1.7320	986	2.0750	0.343
5000	4876	1.9628	4854	1.8731	4898	2.0645	0.191
10000	9751	1.9893	9720	1.9369	9782	2.0548	0.118
100000	97501	1.9638	97404	1.9492	97598	1.9787	0.030
